In [23]:
# 1. Import datasets

# Carregamento do Dataset criado
FOLDER_BASE = "/home/info/MyNotebooks/Datasets/SentencasTRT1/"
DS_FOLDER   = FOLDER_BASE + "DsClassAnot/"
        
DsClassAnot = datasets.load_from_disk(DS_FOLDER)
DsClassAnot

Dataset({
    features: ['text', 'disp', 'label'],
    num_rows: 721
})

In [24]:
# 2.
import pandas as pd

df_anot = pd.DataFrame(DsClassAnot, columns=['text','disp','label'])
df_anot

,text,disp,label
0,PODER JUDICIÁRIO FEDERAL JUSTIÇA DO TRABALHO ...,go de chefe do Poder Executivo estadual assegu...,Procedente
1,Tratase de Ação Civil Pública ajuizada p...,ertinência temática entre os fins sociais da e...,Nenhuma
2,Relatório Vistos SINDICATO DOS EMPREGADOS ...,edidos inexistindo causa de pedir própria rest...,Improcedente
3,PODER JUDICIÁRIO FEDERAL JUSTIÇA DO TRABALHO ...,no evidenciado transmudando a natureza e o des...,Nenhuma
4,Relatório Fundamentação Dispositivo PODER J...,x positis EXTINGUO SEM RESOLUÇÃO DO MÉRITO a p...,Nenhuma
...,...,...,...
716,SENTENÇA PJe RELATÓRIO Tratase de Ação C...,tes do cargo de chefe do Poder Executivo estad...,Procedente
717,PROCESSO 01003540420185010049 S E N T E N Ç ...,or veicula pela presente ação civil pública pr...,Nenhuma
718,PODER JUDICIÁRIO JUSTIÇA DO TRABALHO TRIBUNAL ...,uanto a ponderação denota inequívoco intento r...,Parcialmente procedente
719,PODER JUDICIÁRIO FEDERAL JUSTIÇA DO TRABALHO ...,to da categoria diferenciada e o sindicato que...,Improcedente


In [25]:
df_anot['disp'][8]

'tes do cargo de chefe do Poder Executivo estadual assegurou a manutenção do pagamento queles que já houvessem adquirido o direito de gozar o benefício 3 Há usurpação da competência do STF inscrita no art 102 I a da CF88 quando configurado o ajuizamento de ação civil pública com o intento de dissimular o controle abstrato de constitucionalidade de ato normativo estadual em face da Constituição Federal 4 Arquivamento da ação civil pública ante a ausência de legitimidade ativa ad causam do Parquet estadual para propor ação direta de inconstitucionalidade perante a Suprema Corte nos termos do art 103 da CF88 Precedentes 5 Reclamação julgada procedente para cassar a decisão que julgou procedente o pedido formulado nos autos da ação civil pública declarar a incompetência do juízo de primeira instância e determinar o arquivamento da ação Rcl 19662 Relatora: Min DIAS TOFFOLI Segunda Turma julgado em 06092016 PROCESSO ELETRÔNICO DJe168 DIVULG 31072017 PUBLIC 01082017 sem os destaques no origin

In [26]:
# 3. Split in Train, Validation and Test sets
# https://datascience.stackexchange.com/questions/103249/how-to-use-train-test-split-with-existing-dataset
# https://discuss.huggingface.co/t/split-dataframe-into-validation-and-train-split/16678

from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_anot, test_size = 0.3)
#df_validation, df_test  = train_test_split(temp, test_size = 0.5)

#df_test, df_temp     = train_test_split(test, test_size = 0.9965)

df_train.reset_index(inplace=True)
#df_validation.reset_index(inplace=True)
df_test.reset_index(inplace=True)

print(len(df_train),len(df_test))

504 217


In [27]:
# 4. Remove 'index', 'text', 'classif' column and create a features dataframe/datset
df_train.drop('index', inplace=True, axis=1)
df_test.drop('index', inplace=True, axis=1)

In [28]:
# 5. Cria os Dataset e o DatasetDict e salva em disco (actually MyGoogleDrive)
from datasets import Dataset, DatasetDict

dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)

dataset_target = DatasetDict()
dataset_target['train'] = dataset_train
dataset_target['test'] = dataset_test

#dataset_target.save_to_disk("/content/drive/MyDrive/DenunBertV2")
#DsClassAnot = Dataset.from_pandas(df)
dataset_target.save_to_disk(DS_FOLDER+"DsClassAnot/Train_Test")  # Especificar a pasta DsClassAnot?

In [ ]:
 ### APLICANDO O MODELO DE PeR (Ou Q&A)NO DATASET PARA EXTRAIR A CLASSIFICAÇÃO DA SENTENÇA. ###

In [70]:
# 6. 
import datasets
'''
FOLDER_BASE = "/home/info/MyNotebooks/Datasets/SentencasTRT1/"
DS_FOLDER   = FOLDER_BASE + "DS/"

DsClassAnot = datasets.load_from_disk(DS_FOLDER+"DsClassAnot/Train_Test")
DsClassAnot
'''
# Carregamento do Dataset criado
FOLDER_BASE = "/home/info/MyNotebooks/Datasets/SentencasTRT1/"
DS_FOLDER   = FOLDER_BASE + "DsClassAnot/"
        
DsClassAnot = datasets.load_from_disk(DS_FOLDER)
DsClassAnot

Dataset({
    features: ['text', 'disp', 'label'],
    num_rows: 5900
})

In [71]:
# 7. 
import transformers
from transformers import AutoModelForTokenClassification, AutoModelForQuestionAnswering, AutoTokenizer
from transformers import pipeline

model_name_qa  = "pierreguillou/bert-large-cased-squad-v1.1-portuguese"

model_qa   = AutoModelForQuestionAnswering.from_pretrained(model_name_qa)
tokenz_qa  = AutoTokenizer.from_pretrained(model_name_qa)
qa  = pipeline("question-answering", model=model_qa, tokenizer=tokenz_qa)


In [72]:
# 8.
#
def apply_qa(example):
    questao = "julgo ou decido ou acord?"
    
    output = qa(question=questao, context=example['disp'])   #No disp - Dispositivo ou no text - texto completo da sentença
    
    resposta = output['answer'].lower()
    
    if (("parcialmente procedente" in resposta) or ("procedente em parte" in resposta)):
        resposta = "Parcialmente procedente"

    elif ("improceden" in resposta):
        resposta = "Improcedente"

    elif ("proceden" in resposta):
        resposta = "Procedente"
        
    elif ("acord" in resposta):
        resposta = "Acordo ou outros"
        
    else:
        resposta = "Nenhuma"
    
    example['predicted_labels'] = resposta
    return (example)

In [73]:
# 9.
#DS = DsClassAnot['train']
DS = DsClassAnot
DS_Applied_PeR = DS.map(apply_qa)

  0%|          | 0/5900 [00:00<?, ?ex/s]

In [ ]:
contexto = DsClassAnot['train']['text'][5]
contexto

In [ ]:
contexto = DsClassAnot['train']['text'][1]
questao = "Qual é a Jurisprudência"
resp = qa(question=questao, context=contexto)['answer']
resp

In [ ]:
def apply_qa_v2(example, question, campo):
    
    output = qa(question=question, context=example['text'])   #No disp - Dispositivo ou no text - texto completo da sentença
    
    resposta = output['answer']
       
    example[campo] = resposta
    return (example)

In [ ]:
DS = DsClassAnot['test']
DS_Applied_PeR = DS.map(apply_qa_v2("Quem é Juiz do Trabalho?", "magistrado")

In [75]:
# 10. DS_Applied_PeR['labels']
#DS_Applied_PeR['predicted_labels']

In [76]:
DS_Applied_PeR

Dataset({
    features: ['text', 'disp', 'label', 'predicted_labels'],
    num_rows: 5900
})

In [77]:
# 11.
print("Labels:       | Predicted: \n")
for i in range(len(DS_Applied_PeR)):
    print(DS_Applied_PeR['label'][i],"    | ", DS_Applied_PeR['predicted_labels'][i])

Labels:       | Predicted: 

Parcialmente procedente     |  Parcialmente procedente
Acordo ou outros     |  Nenhuma
Improcedente     |  Nenhuma
Improcedente     |  Improcedente
Nenhuma     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Improcedente     |  Procedente
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Improcedente     |  Improcedente
Acordo ou outros     |  Nenhuma
Procedente     |  Procedente
Procedente     |  Procedente
Improcedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Acordo ou outros     |  Nenhuma
Procedente     |  Procedente
Nenhuma     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Procedente     |  Nenhuma
Procedente     |  Nenhuma
Procedente     |  Nenhuma
Procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Improcedente     |  Nenhuma
Procede

Parcialmente procedente     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Procedente     |  Procedente
Procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Improcedente     |  Improcedente
Acordo ou outros     |  Nenhuma
Improcedente     |  Nenhuma
Procedente     |  Nenhuma
Improcedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Improcedente     |  Procedente
Nenhuma     |  Nenhuma
Procedente     |  Procedente
Nenhuma     |  Nenhuma
Parcialmente procedente     |  Parcialmente procedente
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Improcedente     |  Improcedente
Acordo ou outros     |  Nenhuma
Procedente     |  Procedente
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Procedente
Improcedente     |  Nenhuma
Acordo ou 

Nenhuma     |  Nenhuma
Improcedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Improcedente     |  Nenhuma
Parcialmente procedente     |  Parcialmente procedente
Procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Improcedente     |  Improcedente
Nenhuma     |  Nenhuma
Procedente     |  Procedente
Nenhuma     |  Nenhuma
Improcedente     |  Nenhuma
Improcedente     |  Improcedente
Nenhuma     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Improcedente     |  Nenhuma
Improcedente     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Improcedente     |  Nenhuma
Nenhuma     |  Nenhuma
Improcedente     |  Improcedente
Acordo ou outros     |  Nenhuma
Procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Improcedente     |  Nenhuma
Improcedente     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Improcedente     |  Improcedente
Nenhuma     |  

Parcialmente procedente     |  Nenhuma
Procedente     |  Nenhuma
Improcedente     |  Improcedente
Improcedente     |  Improcedente
Nenhuma     |  Nenhuma
Improcedente     |  Improcedente
Nenhuma     |  Nenhuma
Procedente     |  Nenhuma
Improcedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Nenhuma     |  Nenhuma
Nenhuma     |  Nenhuma
Procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Parcialmente procedente     |  Parcialmente procedente
Procedente     |  Nenhuma
Procedente     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Improcedente     |  Nenhuma
Procedente     |  Nenhuma
Parcialmente procedente     |  Parcialmente procedente
Improcedente     |  Nenhuma
Improcedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Acordo

Nenhuma     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Improcedente     |  Nenhuma
Improcedente     |  Nenhuma
Procedente     |  Nenhuma
Improcedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Procedente     |  Nenhuma
Acordo ou outros     |  Acordo ou outros
Procedente     |  Nenhuma
Procedente     |  Nenhuma
Parcialmente procedente     |  Parcialmente procedente
Acordo ou outros     |  Nenhuma
Improcedente     |  Nenhuma
Nenhuma     |  Nenhuma
Parcialmente procedente     |  Parcialmente procedente
Procedente     |  Procedente
Procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Acordo ou outros     |  Nenhuma
Procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Improcedente     |  Ne

...

Parcialmente procedente     |  Nenhuma
Parcialmente procedente     |  Procedente
Nenhuma     |  Nenhuma
Improcedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Procedente
Improcedente     |  Nenhuma
Improcedente     |  Procedente
Procedente     |  Nenhuma
Improcedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Acordo ou outros     |  Nenhuma
Acordo ou outros     |  Nenhuma
Procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Procedente
Improcedente     |  Improcedente
Nenhuma     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Procedente     |  Procedente
Nenhuma     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Procedente     |  Nenhuma
Nenhuma     |  Nenhuma
P

Acordo ou outros     |  Nenhuma
Improcedente     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Parcialmente procedente     |  Parcialmente procedente
Parcialmente procedente     |  Nenhuma
Procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Parcialmente procedente
Improcedente     |  Nenhuma
Nenhuma     |  Nenhuma
Improcedente     |  Nenhuma
Improcedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Procedente     |  Procedente
Parcialmente procedente     |  Procedente
Procedente     |  Nenhuma
Improcedente     |  Nenhuma
Nenhuma     |  Nenhuma
Procedente     |  Procedente
Procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Improcedente     |  Improcedente
Improcedente     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Procedente     |

Parcialmente procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Improcedente     |  Nenhuma
Improcedente     |  Improcedente
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Improcedente     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Nenhuma     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Improcedente     |  Nenhuma
Improcedente     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Improcedente     |  Nenhuma
Improcedente     |  Improcedente
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Improcedente     |  Nenhuma
Procedente     |  Nenhuma
Procedente     |  Procedent

Improcedente     |  Nenhuma
Procedente     |  Procedente
Nenhuma     |  Nenhuma
Improcedente     |  Nenhuma
Improcedente     |  Nenhuma
Improcedente     |  Nenhuma
Improcedente     |  Improcedente
Parcialmente procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Nenhuma     |  Nenhuma
Improcedente     |  Nenhuma
Nenhuma     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Procedente     |  Procedente
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Improcedente     |  Improcedente
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Nenhuma     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Parcialmente procedente
Procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Improcedente     |  Improced

Parcialmente procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Nenhuma     |  Nenhuma
Improcedente     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Improcedente     |  Nenhuma
Improcedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Procedente     |  Nenhuma
Improcedente     |  Nenhuma
Improcedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Nenhuma     |  Nenhuma
Acordo ou outros     |  Nenhuma
Procedente     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Acordo ou outros     |  Nenhuma
Improcedente     |  Nenhuma
Procedente     |  Nenhuma
Acordo ou outros     |  Nenhuma
Parcialmente procedente     |  Nenhuma
Parcialmente procedente     |  Procedente
Improcedente     |  Nenhuma
Improcedente     |  Improcedente
Procedente     |  Nenhuma
Nenhuma     |  Nenhuma
Nenhuma     |  Nenhuma
Parcialmente procedente     |  Improcedente
Parcialm

In [69]:
# 12.
from sklearn import metrics

print(metrics.classification_report(DS_Applied_PeR['label'], DS_Applied_PeR['predicted_labels'], labels=["Acordo ou outros", "Improcedente", "Procedente","Parcialmente procedente", "Nenhuma"]))



                         precision    recall  f1-score   support

       Acordo ou outros       0.67      0.04      0.08        47
           Improcedente       1.00      0.34      0.50       143
             Procedente       0.76      0.15      0.25       106
Parcialmente procedente       1.00      0.27      0.42        26
                Nenhuma       0.62      1.00      0.77       399

               accuracy                           0.65       721
              macro avg       0.81      0.36      0.41       721
           weighted avg       0.73      0.65      0.58       721



In [78]:
# 12.
from sklearn import metrics

print(metrics.classification_report(DS_Applied_PeR['label'], DS_Applied_PeR['predicted_labels'], labels=["Acordo ou outros", "Improcedente", "Procedente","Parcialmente procedente", "Nenhuma"]))


                         precision    recall  f1-score   support

       Acordo ou outros       0.94      0.01      0.03      1180
           Improcedente       0.93      0.30      0.45      1180
             Procedente       0.70      0.41      0.51      1180
Parcialmente procedente       1.00      0.16      0.27      1180
                Nenhuma       0.25      1.00      0.41      1180

               accuracy                           0.37      5900
              macro avg       0.77      0.37      0.33      5900
           weighted avg       0.77      0.37      0.33      5900



In [ ]:
# Multiclass Classification.
# https://www.programcreek.com/python/example/81623/sklearn.metrics.classification_report